In [156]:
# import

import math
import pymysql 
import requests
from urllib.parse import unquote, urlencode, quote_plus 
from datetime import datetime, date, timedelta

In [44]:
# 기상청 공개 위도 경도 xy 좌표

NX = 149            ## X축 격자점 수
NY = 253            ## Y축 격자점 수

Re = 6371.00877     ##  지도반경
grid = 5.0          ##  격자간격 (km)
slat1 = 30.0        ##  표준위도 1
slat2 = 60.0        ##  표준위도 2
olon = 126.0        ##  기준점 경도
olat = 38.0         ##  기준점 위도
xo = 210 / grid     ##  기준점 X좌표
yo = 675 / grid     ##  기준점 Y좌표
first = 0

if first == 0 :
    PI = math.asin(1.0) * 2.0
    DEGRAD = PI/ 180.0
    RADDEG = 180.0 / PI


    re = Re / grid
    slat1 = slat1 * DEGRAD
    slat2 = slat2 * DEGRAD
    olon = olon * DEGRAD
    olat = olat * DEGRAD

    sn = math.tan(PI * 0.25 + slat2 * 0.5) / math.tan(PI * 0.25 + slat1 * 0.5)
    sn = math.log(math.cos(slat1) / math.cos(slat2)) / math.log(sn)
    sf = math.tan(PI * 0.25 + slat1 * 0.5)
    sf = math.pow(sf, sn) * math.cos(slat1) / sn
    ro = math.tan(PI * 0.25 + olat * 0.5)
    ro = re * sf / math.pow(ro, sn)
    first = 1

def mapToGrid(lat, lon, code = 0 ):
    ra = math.tan(PI * 0.25 + lat * DEGRAD * 0.5)
    ra = re * sf / pow(ra, sn)
    theta = lon * DEGRAD - olon
    if theta > PI :
        theta -= 2.0 * PI
    if theta < -PI :
        theta += 2.0 * PI
    theta *= sn
    x = (ra * math.sin(theta)) + xo
    y = (ro - ra * math.cos(theta)) + yo
    x = int(x + 1.5)
    y = int(y + 1.5)
    return x, y

def gridToMap(x, y, code = 1):
    x = x - 1
    y = y - 1
    xn = x - xo
    yn = ro - y + yo
    ra = math.sqrt(xn * xn + yn * yn)
    if sn < 0.0 :
        ra = -ra
    alat = math.pow((re * sf / ra), (1.0 / sn))
    alat = 2.0 * math.atan(alat) - PI * 0.5
    if math.fabs(xn) <= 0.0 :
        theta = 0.0
    else :
        if math.fabs(yn) <= 0.0 :
            theta = PI * 0.5
            if xn < 0.0 :
                theta = -theta
        else :
            theta = math.atan2(xn, yn)
    alon = theta / sn + olon
    lat = alat * RADDEG
    lon = alon * RADDEG

    return lat, lon

nx, ny = mapToGrid(35.191619912401606, 129.09438859263133)

In [61]:
# 단기예보 (동네예보) 조회 API

'''
require parameter

    serviceKey : 인증키
    numOfRows : 한 페이지 결과 수
    pageNo : 페이지 번호
    dataType : 응답자료형식
    base_date : 발표일자
    base_time : 발표시각
    nx : 예보지점 X 좌표
    ny : 예보지점 Y 좌표

'''

url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst'
serviceKey = 'wGokgRxD1t3z5G4u7MsWumpoCeiWO8JM6yZ87rX1ELTO9nMSUuMOQjHj70rAzuopgyB1iLdKX0S9WK0RLs88bQ%3D%3D'
serviceKeyDecoded = unquote(serviceKey, 'UTF-8')

now = datetime.now()
today = datetime.today().strftime("%Y%m%d")
yesterday = (date.today() - timedelta(days = 1)).strftime("%Y%m%d")

if now.minute<45: # base_time와 base_date 구하는 함수
    if now.hour==0:
        base_time = "2330"
        base_date = yesterday
    else:
        pre_hour = now.hour-1
        if pre_hour<10:
            base_time = "0" + str(pre_hour) + "30"
        else:
            base_time = str(pre_hour) + "30"
        base_date = today
else:
    if now.hour < 10:
        base_time = "0" + str(now.hour) + "30"
    else:
        base_time = str(now.hour) + "30"
    base_date = today

In [93]:
queryParams = '?' + urlencode({ quote_plus('serviceKey') : serviceKeyDecoded, quote_plus('base_date') : base_date, quote_plus('pageNo') : 1,
                                quote_plus('base_time') : base_time, quote_plus('nx') : nx, quote_plus('ny') : ny,
                                quote_plus('dataType') : 'json', quote_plus('numOfRows') : '700'}) #페이지로 안나누고 한번에 받아오기 위해 numOfRows=60으로 설정해주었다
                                

# 값 요청 (웹 브라우저 서버에서 요청 - url주소와 파라미터)
res = requests.get(url + queryParams, verify=False) # verify=False이거 안 넣으면 에러남ㅜㅜ
items = res.json().get('response').get('body').get('items') #데이터들 아이템에 저장
#print(items)# 테스트

In [100]:
weather_data = dict()

for item in items['item']:
    # 날짜/시간 확인
    Date = item['fcstDate']
    Time = item['fcstTime']
    if item['fcstDate'] not in weather_data:
        weather_data[Date] = dict()
    if item['fcstTime'] not in weather_data[Date]:
        weather_data[Date][Time] = dict()
 
    # 예보 값
    # 강수확률 (%)
    if item['category'] == 'POP':
        weather_data[Date][Time]['pop'] = item['fcstValue']
    # 강수형태 
    if item['category'] == 'PTY':
        weather_data[Date][Time]['pty'] = item['fcstValue']
    # 1시간 강수량
    if item['category'] == 'PCP':
        weather_data[Date][Time]['pcp'] = item['fcstValue']
    # 습도
    if item['category'] == 'REH':
        weather_data[Date][Time]['reh'] = item['fcstValue']
    # 1시간 신적설
    if item['category'] == 'SNO':
        weather_data[Date][Time]['sno'] = item['fcstValue']
    # 하늘상태
    if item['category'] == 'SKY':
        weather_data[Date][Time]['sky'] = item['fcstValue']
    # 1시간 기온
    if item['category'] == 'TMP':
        weather_data[Date][Time]['tmp'] = item['fcstValue']
    # 일 최저기온
    if item['category'] == 'TMN':
        weather_data[Date][Time]['tmn'] = item['fcstValue']
    # 일 최고기온
    if item['category'] == 'TMX':
        weather_data[Date][Time]['tmx'] = item['fcstValue']
    # 풍속(동서성분)
    if item['category'] == 'UUU':
        weather_data[Date][Time]['uuu'] = item['fcstValue']
    # 풍속(남북성분)
    if item['category'] == 'VVV':
        weather_data[Date][Time]['vvv'] = item['fcstValue']
    # 파고
    if item['category'] == 'WAV':
        weather_data[Date][Time]['wav'] = item['fcstValue']
    # 풍향
    if item['category'] == 'VEC':
        weather_data[Date][Time]['vec'] = item['fcstValue']
    # 풍속
    if item['category'] == 'WSD':
        weather_data[Date][Time]['wsd'] = item['fcstValue']
    
print("response: ", weather_data)

response:  {'20230316': {'1500': {'tmp': '15', 'uuu': '-1.4', 'vvv': '-0.5', 'vec': '72', 'wsd': '1.6', 'sky': '4', 'pty': '0', 'pop': '30', 'wav': '0', 'pcp': '강수없음', 'reh': '25', 'sno': '적설없음'}, '1600': {'tmp': '15', 'uuu': '-1.6', 'vvv': '-0.5', 'vec': '74', 'wsd': '1.8', 'sky': '1', 'pty': '0', 'pop': '0', 'wav': '0', 'pcp': '강수없음', 'reh': '30', 'sno': '적설없음'}, '1700': {'tmp': '14', 'uuu': '-2.1', 'vvv': '-0.2', 'vec': '85', 'wsd': '2.2', 'sky': '3', 'pty': '0', 'pop': '20', 'wav': '0', 'pcp': '강수없음', 'reh': '35', 'sno': '적설없음'}, '1800': {'tmp': '12', 'uuu': '-1.9', 'vvv': '-0.1', 'vec': '87', 'wsd': '2', 'sky': '3', 'pty': '0', 'pop': '20', 'wav': '0', 'pcp': '강수없음', 'reh': '40', 'sno': '적설없음'}, '1900': {'tmp': '11', 'uuu': '-1.7', 'vvv': '-0.6', 'vec': '72', 'wsd': '1.9', 'sky': '3', 'pty': '0', 'pop': '20', 'wav': '0', 'pcp': '강수없음', 'reh': '50', 'sno': '적설없음'}, '2000': {'tmp': '10', 'uuu': '-1.5', 'vvv': '-1.2', 'vec': '53', 'wsd': '2', 'sky': '3', 'pty': '0', 'pop': '20', 'wav

In [175]:
for logdate in weather_data:
    for j in weather_data[logdate]:
        print(weather_data[logdate][j])

{'tmp': '15', 'uuu': '-1.4', 'vvv': '-0.5', 'vec': '72', 'wsd': '1.6', 'sky': '4', 'pty': '0', 'pop': '30', 'wav': '0', 'pcp': '강수없음', 'reh': '25', 'sno': '적설없음'}
{'tmp': '15', 'uuu': '-1.6', 'vvv': '-0.5', 'vec': '74', 'wsd': '1.8', 'sky': '1', 'pty': '0', 'pop': '0', 'wav': '0', 'pcp': '강수없음', 'reh': '30', 'sno': '적설없음'}
{'tmp': '14', 'uuu': '-2.1', 'vvv': '-0.2', 'vec': '85', 'wsd': '2.2', 'sky': '3', 'pty': '0', 'pop': '20', 'wav': '0', 'pcp': '강수없음', 'reh': '35', 'sno': '적설없음'}
{'tmp': '12', 'uuu': '-1.9', 'vvv': '-0.1', 'vec': '87', 'wsd': '2', 'sky': '3', 'pty': '0', 'pop': '20', 'wav': '0', 'pcp': '강수없음', 'reh': '40', 'sno': '적설없음'}
{'tmp': '11', 'uuu': '-1.7', 'vvv': '-0.6', 'vec': '72', 'wsd': '1.9', 'sky': '3', 'pty': '0', 'pop': '20', 'wav': '0', 'pcp': '강수없음', 'reh': '50', 'sno': '적설없음'}
{'tmp': '10', 'uuu': '-1.5', 'vvv': '-1.2', 'vec': '53', 'wsd': '2', 'sky': '3', 'pty': '0', 'pop': '20', 'wav': '0', 'pcp': '강수없음', 'reh': '55', 'sno': '적설없음'}
{'tmp': '9', 'uuu': '-1.3',

In [195]:
idx = 1

for log_d in weather_data:
    for log_t in weather_data[log_d]:
        conn = pymysql.connect(host='210.119.12.66', user ='root', password = '12345', db = 'miniproject01', charset = 'utf8')
        cur = conn.cursor()
        sql = f'SELECT * FROM ohyurina WHERE log_date = \'{log_d}\' and log_time = \'{log_t}\''
        cur.execute(sql)
        rows = cur.fetchall()
        conn.close()
    
        if len(rows) == 0:
            conn = pymysql.connect(host='210.119.12.66', user ='root', password = '12345', db = 'miniproject01', charset = 'utf8')
            cur = conn.cursor()
            sql = f'INSERT INTO ohyurina(idx, log_date, log_time) VALUES ({idx}, {log_d}, {log_t})'
            cur.execute(sql)
            rows = cur.fetchall()
            conn.commit()
            conn.close()

            idx += 1

IntegrityError: (1062, "Duplicate entry '1' for key 'ohyurina.PRIMARY'")